### Import and Configure API

In [ ]:
!pip install -U -q google.generativeai # Install the Python SDK
!pip install praw
import google.generativeai as genai
import requests
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

import praw

reddit = praw.Reddit(
    client_id='_TkmaQlhemKAQdR1WclVgQ',  # Replace 'YOUR_CLIENT_ID' with your actual client_id
    client_secret='jLCabo1llva3OyrmV9eXqrmVBW56gg',  # Replace 'YOUR_CLIENT_SECRET' with your actual client_secret
    user_agent='my_reddit_scraper'  # e.g., 'my_reddit_scraper'
)

model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.3 MB/s eta 0:00:00


### Functions

In [ ]:
def get_reddit(lst, topic):
    # Join the subreddit list into a format suitable for PRAW
    subreddits = '+'.join(lst)

    # Access the combined subreddit
    combined_subreddit = reddit.subreddit(subreddits)

    # List to hold the titles
    titles = []

    # Search for the term in the combined subreddit and collect titles
    for submission in combined_subreddit.search(topic, limit=10):
        titles.append(submission.title)

    return " ".join(titles)

def get_news(lst, topic):
  # Join the list into a comma-separated string
  sources = ','.join(lst)

  url = (f'https://newsapi.org/v2/everything?'
        f'q={topic}&'
        f'from=2024-05-06&'
        f'sortBy=popularity&'
        f'sources={sources}&'
        f'apiKey=be578ca8b49340fba140d6e73634ca0c')

  response = requests.get(url)
  data = response.json()

  # Initialize an empty string to hold the concatenated text
  concatenated_text = ""

  # Loop through each article in the JSON object
  for article in data["articles"]:
      # Extract the title, description, and content
      title = article.get("title", "")
      description = article.get("description", "")
      content = article.get("content", "")[:-17]

      concatenated_text += f"{title} {description} "

  # Return the concatenated text
  return concatenated_text


def get_prompt(n, topic):
  prompt = 'Please generate ' + str(n) + ' personas that have the most divergent views about ' + topic + '. For each person, provide demographics, psychographics, background story, a list of possible news consumption, and a list of subreddits they might be on.'
  return prompt

def probe_with_media(person_id, question, news_headlines, reddit_headlines):
  response = chat.send_message("How would person " + str(person_id) + " respond to the following question: " + question + " Here are the headlines person " + str(person_id) + " read this week: " + news_headlines + ". Here are the subreddits person " + str(person_id) + " is on: " + reddit_headlines + ". What news headlines or reddit posts influenced their opinions the most?")
  return response.text

def ask_personas(question, media_dict, reddit_dict):
    for (media_id, media_sources), (reddit_id, reddit_sources) in zip(media_dict.items(), reddit_dict.items()):
        print(media_id)
        if media_id == reddit_id:  # Ensuring the keys match, this check can be omitted if keys are guaranteed to match
            news_headlines = get_news(media_sources, topic)
            reddit_headlines = get_reddit(reddit_sources, topic)
            print(probe_with_media(media_id, question, news_headlines, reddit_headlines))



### Generate personas given specified topic

In [ ]:
topic = 'immigration'

In [ ]:
prompt = get_prompt(3, topic)
response = chat.send_message(prompt)
print(response.text)

**Persona 1: The Nationalist**

* **Demographics:** White male, 55+, rural or small town, low education level
* **Psychographics:** Conservative, patriotic, values tradition, distrustful of outsiders
* **Background story:** Grew up in a small town where immigrants were rarely seen. Has seen his community change over time as more immigrants have moved in. Believes that immigrants are taking away jobs and resources from native-born Americans.
* **Possible news consumption:** Fox News, Breitbart News, The Blaze
* **Subreddits:** r/The_Donald, r/Conservative, r/Immigration

**Persona 2: The Humanitarian**

* **Demographics:** White female, 25-44, urban or suburban, college educated
* **Psychographics:** Liberal, compassionate, values diversity, believes in helping those in need
* **Background story:** Grew up in a diverse community and has always been exposed to different cultures. Has volunteered with refugee organizations and believes that immigrants are an asset to the country.
* **Poss

In [ ]:
# response = chat.send_message("Can you change persona 3 to be a 21 year old who went to bryn mawr")
# print(response.text)

### Specify Media Consumption

In [ ]:
media_dict = {1: ['breitbart-news', 'fox-news'],
              2: ['the-washington-post', 'politico'],
              3: ['cnn']}

reddit_dict = {1: ['whiterights', 'conservative'],
               2: ['progressive', 'socialjustice101'],
               3: ['libertarian', 'Anarcho_Capitalism']}

### Probe Personas

In [ ]:
question = 'In your opinion, should immigration policies in your country be more restrictive, more permissive, or remain unchanged?'

In [ ]:
ask_personas(question, media_dict, reddit_dict)

1


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



The headlines and Reddit posts that would most influence Person 1's opinion on immigration policies are:

* **Headlines:**
    * Fed President Kashkari Warns Rates Might Not Be High Enough
    * Blue State Outlook: Almost 1-in-10 New Jersey Residents Are Illegal Aliens
    * Biden Overwhelms Immigration Courts with Over 3.5 Million Cases Pending
    * Bloomberg: Migration Drives Up Housing Costs in 13 Countries
    * 'Irish Lives Matter' -- Thousands Protest Against Mass Migration in Dublin
    * Joe Biden Calls Illegal Aliens 'Model Citizens' While Promoting Amnesty
    * Illegal Alien Accused of Sexually Assaulting 11-Year-Old Girl After Being Freed into U.S. by Biden's DHS
    * Federal Reserve Chairman: Migration Boosts Mortgage Rates
    * Biden takes role as bystander on border and campus protests, surrenders the bully pulpit
    * Illegal migrant accused of snatching young girl had been set loose by Feds
    * Biden plan to extend ObamaCare eligibility to illegal immigrants gets

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



The headlines and Reddit posts that would most influence Person 2's opinion on immigration policies are:

* **Headlines:**
    * Assessing the most important issue to American voters: It’s inflation, not immigration.
    * Biden looks to project presidential power as campaign heats up: From immigration to climate, White House seeks to present an active president despite pushback from Congress and courts.
* **Reddit posts:**
    * Iowa House sends Texas-style immigration enforcement bill to governor
    * Federal judge blocks Trump immigration ban nationwide
    * How will Florida’s new immigration law impact law enforcement?
    * Trump Aide Miller’s Uncle Blasts Him As 'Immigration Hypocrite' In Op-Ed Given That Grandparents Were Immigrants Fleeing Persecution
    * Evangelicals love Trump’s immigration policies.
    * New immigrant families in Denver must exit shelters as the city scales back aid. Many have nowhere to go
    * Michele Bachmann Accidentally Tells the Truth: No Immigra

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



The headlines and Reddit posts that would most influence Person 3's opinion on immigration policies are:

* **Headlines:**
    * Noelia Voigt resigns as Miss USA, citing her mental health
* **Reddit posts:**
    * Different end goals on immigration
    * If an illegal immigrant...
    * Founding Father George Washington on Immigrants from "Shithole" Countries
    * What is your Stance on Immigration from a Libertarian Perspective?
    * In 2024 supporting illegal immigration is supporting the welfare state. I do not support the welfare state.
    * All government immigration restrictions are bad. No exceptions.
    * Illegal immigration
    * We need to end illegal immigration once and for all
    * Libertarian Party of Texas on Twitter, "You can't be pro-free market and anti-immigration."
    * Conservatives on immigration.

These headlines and posts reinforce Person 3's libertarian philosophy of individual liberty and limited government. They also play into his belief that immigratio

### Appendix

In [ ]:
available_news_sources = ['abc-news',
 'al-jazeera-english',
 'axios',
 'bbc-news',
 'bloomberg',
 'breitbart-news',
 'business-insider',
 'business-insider-uk',
 'buzzfeed',
 'cbc-news',
 'cbs-news',
 'cnn',
 'der-tagesspiegel',
 'entertainment-weekly',
 'financial-post',
 'fortune',
 'fox-news',
 'independent',
 'national-review',
 'nbc-news',
 'news24',
 'newsweek',
 'new-york-magazine',
 'politico',
 'reuters',
 'spiegel-online',
 'the-american-conservative',
 'the-globe-and-mail',
 'the-hill',
 'the-huffington-post',
 'the-jerusalem-post',
 'the-verge',
 'the-wall-street-journal',
 'the-washington-post',
 'the-washington-times',
 'time',
 'usa-today',
 'vice-news']

